In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
import string
# from nltk.stem import WordNetLemmatizer
#from stop_words import safe_get_stop_words
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch
#from transformers import BertTokenizer
#from torch.utils.data import Dataset, DataLoader
from torchtext import data
from torch.utils.data import DataLoader, TensorDataset



In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
df=pd.read_csv("/content/drive/MyDrive/hindi_train_val.csv")

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")


In [7]:
df

label                                               text
0          0                      भीड़ में  बहुत  हब्सी मिलेंगे
1          0          साले बेवकूफ अपनी मां मक्खियां  तो हटा दें
2          0                   बुर देदो तो मुह में लंड ले लो तो
3          0               कुत्ता वहा है चिल्ला तू क्यों रहा है
4          1          चाय नहीं पीता हूं मैं इसी को छोड़ दिया ok
...      ...                                                ...
20178      0             और छोटे भी क्यु पहनते हो, कहा जरूरत है
20179      1             कैसी हो सोना क्या आप हमसे दोस्ती करोगी
20180      1          इस से ये पता चलता है अब चुनाव आ रहा है🙄🙄🙄
20181      1                    खेत जोते हुए किसान भैंस के साथो
20182      0  काश आगर यह इतना दिमाग पढाई में लगायें तो शायद ...

[20183 rows x 2 columns]

In [8]:
text = df.text.values
labels = df.label.values

In [9]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)

Loading BERT tokenizer...


In [10]:
# Print the original sentence.
print(' Original: ', text[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(text[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[0])))

 Original:  भीड़ में  बहुत  हब्सी मिलेंगे
Tokenized:  ['भी', '##ड', 'म', 'ब', '##हत', 'ह', '##ब', '##सी', 'म', '##िल', '##ग']
Token IDs:  [13286, 20691, 889, 887, 108775, 899, 18351, 27117, 889, 27391, 19741]


In [11]:
from sklearn.model_selection import train_test_split

X = text
y = labels

# Split the data into a training set and a validation set
X_train_text, X_val_text, y_train_text, y_val_text = train_test_split(X, y, test_size=0.1, random_state=42)

In [12]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in text:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', text[0])
print('Token IDs:', input_ids[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (572 > 512). Running this sequence through the model will result in indexing errors


Original:  भीड़ में  बहुत  हब्सी मिलेंगे
Token IDs: [101, 13286, 20691, 889, 887, 108775, 899, 18351, 27117, 889, 27391, 19741, 102]


In [13]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  590


In [17]:
MAX_LEN = 64


def pad_or_truncate(tokens, max_len):
    if len(tokens) >= max_len:
        
        return tokens[:max_len]
    else:
        
        return tokens + [0] * (max_len - len(tokens))


input_ids = [pad_or_truncate(ids, MAX_LEN) for ids in input_ids]


In [18]:

attention_masks = []


for sent in input_ids:
    
   
    att_mask = [int(token_id > 0) for token_id in sent]
    
    
    attention_masks.append(att_mask)

In [19]:

from sklearn.model_selection import train_test_split


train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

In [20]:
# Convert all inputs and labels into torch tensors, the required datatype 

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [21]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


batch_size = 32


train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [22]:
from transformers import BertForSequenceClassification, AdamW, BertConfig


model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", 
    num_labels = 2, 
    output_attentions = False, 
    output_hidden_states = False, 
)


model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [23]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (119547, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

In [24]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
from transformers import get_linear_schedule_with_warmup


epochs = 3


total_steps = len(train_dataloader) * epochs


scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [31]:
import numpy as np


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [32]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
   
    elapsed_rounded = int(round((elapsed)))
    
   
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [33]:
import torch


if torch.cuda.is_available():    

    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))


else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [35]:
import random



seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

   
    model.train()

    
    for step, batch in enumerate(train_dataloader):

       
        if step % 40 == 0 and not step == 0:
            
            elapsed = format_time(time.time() - t0)
            
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        
        model.zero_grad()        

        
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
       
        loss = outputs[0]

        total_loss += loss.item()

        
        loss.backward()

       
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        
        optimizer.step()

        
        scheduler.step()

    
    avg_train_loss = total_loss / len(train_dataloader)            
    
    
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    
    model.eval()

    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

   
    for batch in validation_dataloader:
        
        
        batch = tuple(t.to(device) for t in batch)
        
        
        b_input_ids, b_input_mask, b_labels = batch
        
       
        with torch.no_grad():        

            
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
       
        logits = outputs[0]

        
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        
        eval_accuracy += tmp_eval_accuracy

        
        nb_eval_steps += 1

    
    print("  Accuracy: {0:.10f}".format(eval_accuracy/nb_eval_steps*100))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training...
  Batch    40  of    568.    Elapsed: 0:00:15.
  Batch    80  of    568.    Elapsed: 0:00:31.
  Batch   120  of    568.    Elapsed: 0:00:46.
  Batch   160  of    568.    Elapsed: 0:01:01.
  Batch   200  of    568.    Elapsed: 0:01:16.
  Batch   240  of    568.    Elapsed: 0:01:30.
  Batch   280  of    568.    Elapsed: 0:01:45.
  Batch   320  of    568.    Elapsed: 0:02:00.
  Batch   360  of    568.    Elapsed: 0:02:15.
  Batch   400  of    568.    Elapsed: 0:02:30.
  Batch   440  of    568.    Elapsed: 0:02:45.
  Batch   480  of    568.    Elapsed: 0:03:00.
  Batch   520  of    568.    Elapsed: 0:03:15.
  Batch   560  of    568.    Elapsed: 0:03:30.

  Average training loss: 0.27
  Training epcoh took: 0:03:33

Running Validation...
  Accuracy: 79.6061197917
  Validation took: 0:00:08

======== Epoch 2 / 3 ========
Training...
  Batch    40  of    568.    Elapsed: 0:00:15.
  Batch    80  of    568.    Elapsed: 0:00:30.
  Batch   120  of    568

FOR TESTING
(UPLOAD THE TRAIN AND TEST FILE IN HOME FOLDER OF DRIVE AND CHANGE FILENAME ACCORDINGLY)


In [36]:
df=pd.read_csv("/content/drive/MyDrive/hindi_test.csv")

In [72]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [73]:
text = df.text.values
labels = df.label.values

In [74]:
# Print the original sentence.
print(' Original: ', text[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(text[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[0])))

 Original:  मैं ये नही सोच रहा की इसे निकले कैसे मैं ये सोच रहा की ये फंसा कैसे😂😂😂
Tokenized:  ['म', 'य', 'नही', 'स', '##ो', '##च', 'रहा', 'की', 'इस', 'न', '##िकल', 'क', '##स', 'म', 'य', 'स', '##ो', '##च', 'रहा', 'की', 'य', 'फस', '##ा', '[UNK]']
Token IDs:  [571, 572, 17024, 579, 11231, 14870, 36335, 10944, 13901, 566, 79890, 547, 12114, 571, 572, 579, 11231, 14870, 36335, 10944, 572, 60856, 10949, 100]


In [103]:
from sklearn.model_selection import train_test_split

X = text
y = labels

# Split the data into a training set and a validation set
X_train_text_test, X_val_text_test, y_train_text, y_val_text = train_test_split(X, y, test_size=0.2, random_state=42)

In [76]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in text:
    
    encoded_sent = tokenizer.encode(# Number of training epochs (authors recommend between 2 and 4)

                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', text[0])
print('Token IDs:', input_ids[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (546 > 512). Running this sequence through the model will result in indexing errors


Original:  मैं ये नही सोच रहा की इसे निकले कैसे मैं ये सोच रहा की ये फंसा कैसे😂😂😂
Token IDs: [101, 571, 572, 17024, 579, 11231, 14870, 36335, 10944, 13901, 566, 79890, 547, 12114, 571, 572, 579, 11231, 14870, 36335, 10944, 572, 60856, 10949, 100, 102]


In [77]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  546


In [78]:
MAX_LEN = 64

# Define a function to pad or truncate a list of tokens to a specified length
def pad_or_truncate(tokens, max_len):
    if len(tokens) >= max_len:
        # Truncate the list if it's longer than max_len
        return tokens[:max_len]
    else:
        # Pad the list with zeros if it's shorter than max_len
        return tokens + [0] * (max_len - len(tokens))

# Apply the function to each list of input_ids
input_ids = [pad_or_truncate(ids, MAX_LEN) for ids in input_ids]


In [79]:

attention_masks = []


for sent in input_ids:
    
   
    att_mask = [int(token_id > 0) for token_id in sent]
    
    
    attention_masks.append(att_mask)

In [80]:
from sklearn.model_selection import train_test_split


train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)


In [81]:

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [82]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler



batch_size = 32


train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)


validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [83]:
from transformers import BertForSequenceClassification, AdamW, BertConfig


model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", 
    num_labels = 2, 
                    
    output_attentions = False,
    output_hidden_states = False, 
)


model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [84]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (119547, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

In [85]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )


In [96]:
from transformers import get_linear_schedule_with_warmup

epochs = 4

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [97]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [98]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    
    elapsed_rounded = int(round((elapsed)))
    
    
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [99]:
import torch

if torch.cuda.is_available():    

   
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))


else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [100]:
import random



# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


loss_values = []


for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

   
    t0 = time.time()

    
    total_loss = 0

    
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        
        if step % 40 == 0 and not step == 0:
            
            elapsed = format_time(time.time() - t0)
            
           
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        
        model.zero_grad()        

        
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        
        loss = outputs[0]

       
        total_loss += loss.item()

        
        loss.backward()

       
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

   
    avg_train_loss = total_loss / len(train_dataloader)            
    
    
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    
    model.eval()

   
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    
    for batch in validation_dataloader:
        
        
        batch = tuple(t.to(device) for t in batch)
        
        
        b_input_ids, b_input_mask, b_labels = batch
        
        
        with torch.no_grad():        

           
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        
        logits = outputs[0]

        
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
       
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        
        eval_accuracy += tmp_eval_accuracy

        
        nb_eval_steps += 1

   
    print("  Accuracy: {0:.10f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    190.    Elapsed: 0:00:15.
  Batch    80  of    190.    Elapsed: 0:00:31.
  Batch   120  of    190.    Elapsed: 0:00:46.
  Batch   160  of    190.    Elapsed: 0:01:01.

  Average training loss: 0.13
  Training epcoh took: 0:01:11

Running Validation...
  Accuracy: 0.6889204545
  Validation took: 0:00:03

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    190.    Elapsed: 0:00:15.
  Batch    80  of    190.    Elapsed: 0:00:30.
  Batch   120  of    190.    Elapsed: 0:00:45.
  Batch   160  of    190.    Elapsed: 0:01:00.

  Average training loss: 0.11
  Training epcoh took: 0:01:11

Running Validation...
  Accuracy: 0.7116477273
  Validation took: 0:00:03

======== Epoch 3 / 4 ========
Training...
  Batch    40  of    190.    Elapsed: 0:00:15.
  Batch    80  of    190.    Elapsed: 0:00:30.
  Batch   120  of    190.    Elapsed: 0:00:44.
  Batch   160  of    190.    Elapsed: 0:00:59.

  Average training loss: 0.07
  

INTERSECTION BETWEEN TRAIN VAL AND TEST SET

In [114]:

intersect = np.intersect1d(X_train_text, X_train_text_test)
print(intersect)

[]


In [115]:
intersect2=np.intersect1d(X_val_text, X_train_text_test)
print(intersect2)
print(f'No of common texts:{len(intersect2)}')

[]
No of common texts:0
